<b><center> Homework 4 </center></b>
<br>
<b><center> James Bonifield </center></b>
<br>
<br>
<b>Question 7.1</b><br>
Describe a situation or problem from your job, everyday life, current events, etc., for which exponential
smoothing would be appropriate. What data would you need? Would you expect the value of $\alpha$ (the
first smoothing parameter) to be closer to 0 or 1, and why? <br> <br>
<i> In my current role as a technical consultant for a software company, I often need to review usage logs and draw conclusions from noisy performance data taken at the minute or even second level of detail where a smoothing model would be very applicable to reduce said noise. I would likely chose a large to mid sized $\alpha$ value, since I want to be able to detect peaks without too much smoothing that a low $\alpha$ value might overwrite.</i>

<b>Question 7.2</b><br>
Using the 20 years of daily high temperature data for Atlanta (July through October) from Question 6.2,
build and use an exponential smoothing model to help make a judgment of whether the unofficial end
of summer has gotten later over the 20 years. (Part of the point of this assignment is for you to think
about how you might use exponential smoothing to answer this question. Feel free to combine it with
other models if you’d like to. There’s certainly more than one reasonable approach.)

In [1]:
library(tidyverse)
library(lubridate)
TempData <- suppressMessages(read_tsv("temps.txt"))
print(TempData)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
<U+221A> ggplot2 2.2.1     <U+221A> purrr   0.2.4
<U+221A> tibble  1.4.1     <U+221A> dplyr   0.7.4
<U+221A> tidyr   0.7.2     <U+221A> stringr 1.2.0
<U+221A> readr   1.1.1     <U+221A> forcats 0.2.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date



# A tibble: 123 x 21
   DAY    `1996` `1997` `1998` `1999` `2000` `2001` `2002` `2003` `2004` `2005`
   <chr>   <int>  <int>  <int>  <int>  <int>  <int>  <int>  <int>  <int>  <int>
 1 1-Jul      98     86     91     84     89     84     90     73     82     91
 2 2-Jul      97     90     88     82     91     87     90     81     81     89
 3 3-Jul      97     93     91     87     93     87     87     87     86     86
 4 4-Jul      90     91     91     88     95     84     89     86     88     86
 5 5-Jul      89     84     91     90     96     86     93     80     90     89
 6 6-Jul      93     84     89     91     96     87     93     84     90     82
 7 7-Jul      93     75     93     82     96     87     89     87     89     76
 8 8-Jul      91     87     95     86     91     89     89     90     87     88
 9 9-Jul      93     84     95     87     96     91     90     89     88     89
10 10-Jul     93     87     91     87     99     87     91     84     89     78
# ... with 113 more

In [174]:
#Below is my attempt to 'tidyverse' data into a ts. It didn't really work that well, so you can just
#Skip down to the tsData info
FlatTempData <- as_tibble(gather(TempData, '1996','1997','1998','1999','2000','2001',
                    '2002','2003','2004','2005','2006','2007','2008','2009',
                    '2010','2011','2012','2013','2014','2015', key = 'Year', value = 'Temps'))
FlatTempData <- mutate(FlatTempData, DAY= dmy(paste(DAY,Year,sep = '-')))
FlatTempData$Year <- NULL
FlatTempData$DAY <- NULL

#Convert to Time Series
tsData <- ts(FlatTempData,start=1996, frequency=123)


First, let's take a look at the Double Exponential Smoothing Model (no seasonality) to get an idea of the trend

In [202]:
#Generate Model
?HoltWinters
hwDoubleExpModel <- HoltWinters(tsData, gamma=FALSE)
hwDoubleExpModel

Holt-Winters exponential smoothing with trend and without seasonal component.

Call:
HoltWinters(x = tsData, gamma = FALSE, seasonal = "multiplicative")

Smoothing parameters:
 alpha: 0.8445729
 beta : 0.003720884
 gamma: FALSE

Coefficients:
        [,1]
a 63.2530022
b -0.0729933

From these results we see that the trend (b) is almost zero and can pretty much be ignored in our analysis for seasonality (I think, not 100% on this one - let me know in the comments) <br>
Now we take a look at the data with seasonality, and store the seasonality in a 123x19 matrix (days x years) to run CUSUM on each day of the year.

In [222]:
hwSeasonalNoTrend <-HoltWinters(tsData, seasonal='multiplicative')
hwSeasonalNoTrend
SeasonalData <- as.tibble(matrix(hwSeasonalNoTrend$fitted[,4],ncol=19))

Holt-Winters exponential smoothing with trend and multiplicative seasonal component.

Call:
HoltWinters(x = tsData, seasonal = "multiplicative")

Smoothing parameters:
 alpha: 0.615003
 beta : 0
 gamma: 0.5495256

Coefficients:
             [,1]
a    73.679517064
b    -0.004362918
s1    1.239022317
s2    1.234344062
s3    1.159509551
s4    1.175247483
s5    1.171344196
s6    1.151038408
s7    1.139383104
s8    1.130484528
s9    1.110487514
s10   1.076242879
s11   1.041044609
s12   1.058139281
s13   1.032496529
s14   1.036257448
s15   1.019348815
s16   1.026754142
s17   1.071170378
s18   1.054819556
s19   1.084397734
s20   1.064605879
s21   1.109827336
s22   1.112670130
s23   1.103970506
s24   1.102771209
s25   1.091264692
s26   1.084518342
s27   1.077914660
s28   1.077696145
s29   1.053788854
s30   1.079454300
s31   1.053481186
s32   1.054023885
s33   1.078221405
s34   1.070145761
s35   1.054891375
s36   1.044587771
s37   1.023285461
s38   1.025836722
s39   1.031075732
s40   1.03141915

Writing the data to CSV:

In [223]:
write_csv(SeasonalData,path='Seasonal_Data.csv')

Analysing the data in Excel with two CUSUM analysis (one for each tail),I found no discernable difference between the early and later year cohorts of seasonality, and conclude like in the previous assignment that this data does not suggest one way or another about changing summer dates or temperatures.